In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from dateutil import relativedelta

In [2]:
import os
os.chdir("../")

In [3]:
matplotlib.rcParams['font.sans-serif'] = ['SimSun'] 
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

#### 1.数据导入

In [4]:
# 个股的拥挤度->行业拥挤度
StkClose = pd.read_csv("./Data/StkClose.csv",index_col = 0)
StkPe = pd.read_csv("./Data/StkPe.csv",index_col = 0)
StkVolume = pd.read_csv("./Data/StkVolume.csv",index_col = 0)

In [5]:
StkClose.ffill(axis = 0,inplace = True)
StkPe.ffill(axis = 0,inplace = True)
StkVolume.replace(0,np.nan,inplace = True)

In [6]:
StkClose.index = pd.to_datetime(StkClose.index)
StkPe.index = pd.to_datetime(StkPe.index)
StkVolume.index = pd.to_datetime(StkVolume.index)

In [7]:
start_date = pd.to_datetime("2013-12-31")
end_date = pd.to_datetime("2022-04-15")

In [8]:
StkClose = StkClose.loc[start_date:end_date,:]
StkPe = StkPe.loc[start_date:end_date,:]
StkVolume = StkVolume.loc[start_date:end_date,:]

In [9]:
CITICIndustryName = pd.read_excel("./Data/CITICIndustryName.xlsx")
SktListedDate = pd.read_excel("./Data/SktListedDate.xlsx")
StkIndustry = pd.read_excel("./Data/StkIndustry.xlsx")

In [10]:
CITICIndustryIndexClose = pd.read_excel("./Data/CITICIndustryIndexClose.xlsx",index_col = 0)
CITICIndustryIndexClose.index = pd.to_datetime(CITICIndustryIndexClose.index)
CITICIndustryIndexClose = CITICIndustryIndexClose.loc[start_date:end_date,:]

In [11]:
for stk in StkClose.columns:
    stk_listed_date = pd.to_datetime(SktListedDate.query(f"S_INFO_WINDCODE == '{stk}'").LISTED_DATE.values[0])
    StkClose.loc[:stk_listed_date - relativedelta.relativedelta(days = 1),stk] = np.nan
    StkPe.loc[:stk_listed_date - relativedelta.relativedelta(days = 1),stk] = np.nan
    StkVolume.loc[:stk_listed_date - relativedelta.relativedelta(days = 1),stk] = np.nan

#### 2.求解个股马氏距离

In [12]:
stk_mahalanobis_distance = pd.DataFrame(index = StkClose.index, columns = StkClose.columns)

In [13]:
for date in StkClose.index:
    print(f"processing {date}")
    for stk in StkClose.columns:
        period_close = StkClose.loc[:date,stk].iloc[-60:].dropna()
        period_pe = StkPe.loc[:date,stk].iloc[-60:].dropna()
        period_volume = StkVolume.loc[:date,stk].iloc[-60:].dropna() # 只要没成交量便没上市
        if len(period_close)<60 or len(period_pe)<60 or len(period_volume)<60:
            continue
        
        dist = np.array([period_close.iloc[-1],period_pe.iloc[-1],period_volume.iloc[-1]])-\
        np.array([period_close.mean(),period_pe.mean(),period_volume.mean()]) # mean
        
        cov = np.cov([period_close,period_pe,period_volume])
        cov_inverse = np.linalg.inv(cov)
        mah_dist = dist@cov_inverse@np.transpose(dist)
        stk_mahalanobis_distance.at[date,stk] = mah_dist

processing 2013-12-31 00:00:00
processing 2014-01-02 00:00:00
processing 2014-01-03 00:00:00
processing 2014-01-06 00:00:00
processing 2014-01-07 00:00:00
processing 2014-01-08 00:00:00
processing 2014-01-09 00:00:00
processing 2014-01-10 00:00:00
processing 2014-01-13 00:00:00
processing 2014-01-14 00:00:00
processing 2014-01-15 00:00:00
processing 2014-01-16 00:00:00
processing 2014-01-17 00:00:00
processing 2014-01-20 00:00:00
processing 2014-01-21 00:00:00
processing 2014-01-22 00:00:00
processing 2014-01-23 00:00:00
processing 2014-01-24 00:00:00
processing 2014-01-27 00:00:00
processing 2014-01-28 00:00:00
processing 2014-01-29 00:00:00
processing 2014-01-30 00:00:00
processing 2014-02-07 00:00:00
processing 2014-02-10 00:00:00
processing 2014-02-11 00:00:00
processing 2014-02-12 00:00:00
processing 2014-02-13 00:00:00
processing 2014-02-14 00:00:00
processing 2014-02-17 00:00:00
processing 2014-02-18 00:00:00
processing 2014-02-19 00:00:00
processing 2014-02-20 00:00:00
processi

processing 2015-01-30 00:00:00
processing 2015-02-02 00:00:00
processing 2015-02-03 00:00:00
processing 2015-02-04 00:00:00
processing 2015-02-05 00:00:00
processing 2015-02-06 00:00:00
processing 2015-02-09 00:00:00
processing 2015-02-10 00:00:00
processing 2015-02-11 00:00:00
processing 2015-02-12 00:00:00
processing 2015-02-13 00:00:00
processing 2015-02-16 00:00:00
processing 2015-02-17 00:00:00
processing 2015-02-25 00:00:00
processing 2015-02-26 00:00:00
processing 2015-02-27 00:00:00
processing 2015-03-02 00:00:00
processing 2015-03-03 00:00:00
processing 2015-03-04 00:00:00
processing 2015-03-05 00:00:00
processing 2015-03-06 00:00:00
processing 2015-03-09 00:00:00
processing 2015-03-10 00:00:00
processing 2015-03-11 00:00:00
processing 2015-03-12 00:00:00
processing 2015-03-13 00:00:00
processing 2015-03-16 00:00:00
processing 2015-03-17 00:00:00
processing 2015-03-18 00:00:00
processing 2015-03-19 00:00:00
processing 2015-03-20 00:00:00
processing 2015-03-23 00:00:00
processi

processing 2016-03-07 00:00:00
processing 2016-03-08 00:00:00
processing 2016-03-09 00:00:00
processing 2016-03-10 00:00:00
processing 2016-03-11 00:00:00
processing 2016-03-14 00:00:00
processing 2016-03-15 00:00:00
processing 2016-03-16 00:00:00
processing 2016-03-17 00:00:00
processing 2016-03-18 00:00:00
processing 2016-03-21 00:00:00
processing 2016-03-22 00:00:00
processing 2016-03-23 00:00:00
processing 2016-03-24 00:00:00
processing 2016-03-25 00:00:00
processing 2016-03-28 00:00:00
processing 2016-03-29 00:00:00
processing 2016-03-30 00:00:00
processing 2016-03-31 00:00:00
processing 2016-04-01 00:00:00
processing 2016-04-05 00:00:00
processing 2016-04-06 00:00:00
processing 2016-04-07 00:00:00
processing 2016-04-08 00:00:00
processing 2016-04-11 00:00:00
processing 2016-04-12 00:00:00
processing 2016-04-13 00:00:00
processing 2016-04-14 00:00:00
processing 2016-04-15 00:00:00
processing 2016-04-18 00:00:00
processing 2016-04-19 00:00:00
processing 2016-04-20 00:00:00
processi

processing 2017-04-07 00:00:00
processing 2017-04-10 00:00:00
processing 2017-04-11 00:00:00
processing 2017-04-12 00:00:00
processing 2017-04-13 00:00:00
processing 2017-04-14 00:00:00
processing 2017-04-17 00:00:00
processing 2017-04-18 00:00:00
processing 2017-04-19 00:00:00
processing 2017-04-20 00:00:00
processing 2017-04-21 00:00:00
processing 2017-04-24 00:00:00
processing 2017-04-25 00:00:00
processing 2017-04-26 00:00:00
processing 2017-04-27 00:00:00
processing 2017-04-28 00:00:00
processing 2017-05-02 00:00:00
processing 2017-05-03 00:00:00
processing 2017-05-04 00:00:00
processing 2017-05-05 00:00:00
processing 2017-05-08 00:00:00
processing 2017-05-09 00:00:00
processing 2017-05-10 00:00:00
processing 2017-05-11 00:00:00
processing 2017-05-12 00:00:00
processing 2017-05-15 00:00:00
processing 2017-05-16 00:00:00
processing 2017-05-17 00:00:00
processing 2017-05-18 00:00:00
processing 2017-05-19 00:00:00
processing 2017-05-22 00:00:00
processing 2017-05-23 00:00:00
processi

processing 2018-05-09 00:00:00
processing 2018-05-10 00:00:00
processing 2018-05-11 00:00:00
processing 2018-05-14 00:00:00
processing 2018-05-15 00:00:00
processing 2018-05-16 00:00:00
processing 2018-05-17 00:00:00
processing 2018-05-18 00:00:00
processing 2018-05-21 00:00:00
processing 2018-05-22 00:00:00
processing 2018-05-23 00:00:00
processing 2018-05-24 00:00:00
processing 2018-05-25 00:00:00
processing 2018-05-28 00:00:00
processing 2018-05-29 00:00:00
processing 2018-05-30 00:00:00
processing 2018-05-31 00:00:00
processing 2018-06-01 00:00:00
processing 2018-06-04 00:00:00
processing 2018-06-05 00:00:00
processing 2018-06-06 00:00:00
processing 2018-06-07 00:00:00
processing 2018-06-08 00:00:00
processing 2018-06-11 00:00:00
processing 2018-06-12 00:00:00
processing 2018-06-13 00:00:00
processing 2018-06-14 00:00:00
processing 2018-06-15 00:00:00
processing 2018-06-19 00:00:00
processing 2018-06-20 00:00:00
processing 2018-06-21 00:00:00
processing 2018-06-22 00:00:00
processi

processing 2019-06-11 00:00:00
processing 2019-06-12 00:00:00
processing 2019-06-13 00:00:00
processing 2019-06-14 00:00:00
processing 2019-06-17 00:00:00
processing 2019-06-18 00:00:00
processing 2019-06-19 00:00:00
processing 2019-06-20 00:00:00
processing 2019-06-21 00:00:00
processing 2019-06-24 00:00:00
processing 2019-06-25 00:00:00
processing 2019-06-26 00:00:00
processing 2019-06-27 00:00:00
processing 2019-06-28 00:00:00
processing 2019-07-01 00:00:00
processing 2019-07-02 00:00:00
processing 2019-07-03 00:00:00
processing 2019-07-04 00:00:00
processing 2019-07-05 00:00:00
processing 2019-07-08 00:00:00
processing 2019-07-09 00:00:00
processing 2019-07-10 00:00:00
processing 2019-07-11 00:00:00
processing 2019-07-12 00:00:00
processing 2019-07-15 00:00:00
processing 2019-07-16 00:00:00
processing 2019-07-17 00:00:00
processing 2019-07-18 00:00:00
processing 2019-07-19 00:00:00
processing 2019-07-22 00:00:00
processing 2019-07-23 00:00:00
processing 2019-07-24 00:00:00
processi

processing 2020-07-13 00:00:00
processing 2020-07-14 00:00:00
processing 2020-07-15 00:00:00
processing 2020-07-16 00:00:00
processing 2020-07-17 00:00:00
processing 2020-07-20 00:00:00
processing 2020-07-21 00:00:00
processing 2020-07-22 00:00:00
processing 2020-07-23 00:00:00
processing 2020-07-24 00:00:00
processing 2020-07-27 00:00:00
processing 2020-07-28 00:00:00
processing 2020-07-29 00:00:00
processing 2020-07-30 00:00:00
processing 2020-07-31 00:00:00
processing 2020-08-03 00:00:00
processing 2020-08-04 00:00:00
processing 2020-08-05 00:00:00
processing 2020-08-06 00:00:00
processing 2020-08-07 00:00:00
processing 2020-08-10 00:00:00
processing 2020-08-11 00:00:00
processing 2020-08-12 00:00:00
processing 2020-08-13 00:00:00
processing 2020-08-14 00:00:00
processing 2020-08-17 00:00:00
processing 2020-08-18 00:00:00
processing 2020-08-19 00:00:00
processing 2020-08-20 00:00:00
processing 2020-08-21 00:00:00
processing 2020-08-24 00:00:00
processing 2020-08-25 00:00:00
processi

processing 2021-08-11 00:00:00
processing 2021-08-12 00:00:00
processing 2021-08-13 00:00:00
processing 2021-08-16 00:00:00
processing 2021-08-17 00:00:00
processing 2021-08-18 00:00:00
processing 2021-08-19 00:00:00
processing 2021-08-20 00:00:00
processing 2021-08-23 00:00:00
processing 2021-08-24 00:00:00
processing 2021-08-25 00:00:00
processing 2021-08-26 00:00:00
processing 2021-08-27 00:00:00
processing 2021-08-30 00:00:00
processing 2021-08-31 00:00:00
processing 2021-09-01 00:00:00
processing 2021-09-02 00:00:00
processing 2021-09-03 00:00:00
processing 2021-09-06 00:00:00
processing 2021-09-07 00:00:00
processing 2021-09-08 00:00:00
processing 2021-09-09 00:00:00
processing 2021-09-10 00:00:00
processing 2021-09-13 00:00:00
processing 2021-09-14 00:00:00
processing 2021-09-15 00:00:00
processing 2021-09-16 00:00:00
processing 2021-09-17 00:00:00
processing 2021-09-22 00:00:00
processing 2021-09-23 00:00:00
processing 2021-09-24 00:00:00
processing 2021-09-27 00:00:00
processi

In [14]:
stk_mahalanobis_distance.to_excel("./Output/StkMeanMahalanobisDist.xlsx")

#### 3.合成行业马氏距离

In [15]:
stk_mahalanobis_distance = pd.read_excel("./Output/StkMeanMahalanobisDist.xlsx",index_col = 0)

In [16]:
industry_mahalanobis_distance = pd.DataFrame(index = stk_mahalanobis_distance.index,columns = CITICIndustryName.S_INFO_WINDCODE,data = 0)

In [17]:
CITICIndustryName.S_INFO_WINDNAME = CITICIndustryName.S_INFO_WINDNAME.map(lambda x:x.replace("(中信)",""))

In [19]:
for industry in CITICIndustryName.S_INFO_WINDNAME:
    print(f"processing {industry}")
    industry_index = CITICIndustryName.query(f"S_INFO_WINDNAME == '{industry}'").S_INFO_WINDCODE.tolist()
    if industry in ["保险Ⅱ","证券Ⅱ"]:
        stk_group = StkIndustry.query("CITIC_SECOND_INDUSTRY == '{}'".format(industry)).S_INFO_WINDCODE.tolist()
    elif industry == "水泥Ⅲ":
        industry = "水泥"
        stk_group = StkIndustry.query("CITIC_THIRD_INDUSTRY == '{}'".format(industry)).S_INFO_WINDCODE.tolist()
    else:
        stk_group = StkIndustry.query("CITIC_FIRST_INDUSTRY == '{}'".format(industry)).S_INFO_WINDCODE.tolist()
    for date in stk_mahalanobis_distance.index:
        industry_mahalanobis_distance.at[date,industry_index[0]] = stk_mahalanobis_distance.loc[date,stk_group].dropna().mean()

processing 石油石化
processing 煤炭
processing 电力及公用事业
processing 建材
processing 机械
processing 轻工制造
processing 建筑
processing 基础化工
processing 钢铁
processing 有色金属
processing 计算机
processing 传媒
processing 通信
processing 电子
processing 交通运输
processing 房地产
processing 银行
processing 农林牧渔
processing 食品饮料
processing 医药
processing 纺织服装
processing 家电
processing 消费者服务
processing 商贸零售
processing 汽车
processing 国防军工
processing 电力设备及新能源
processing 水泥Ⅲ
processing 保险Ⅱ
processing 证券Ⅱ


#### 4.作图查看是否具有卖出预警效果

In [20]:
for i in industry_mahalanobis_distance.columns:
    print(industry_mahalanobis_distance.loc[:,i].mean())

5.622312702263513
5.664410172083822
5.626430029803807
5.658217933870428
5.624926414375041
5.613476193340694
5.60968158221686
5.645526407600764
5.679436959662139
5.722838420096748
5.6288865018338905
5.6221509134161165
5.6349379391341685
5.648584875610122
5.656794264468942
5.677582552849182
5.628733319110836
5.631013562088232
5.585521526214529
5.629060579511505
5.540177329677822
5.608793900684032
5.622126018663484
5.649732607165524
5.652422433360654
5.656760808919623
5.664269792330357
5.632699197728726
5.808500928651279
5.822756424302724


In [21]:
for tmp_industry in CITICIndustryIndexClose.columns:
    plt.figure(figsize = (20,7),dpi = 200)
    plt.plot(CITICIndustryIndexClose.loc[:,tmp_industry], "-",color = "#0000FF",linewidth = 0.8,label = "Net Value")
    for date in CITICIndustryIndexClose.index:
        if industry_mahalanobis_distance.at[date,tmp_industry] > 15:
            plt.axvline(pd.to_datetime(date),linestyle = '-',color = "red",linewidth = 0.5)
    plt.xticks(fontproperties='Times New Roman', size = 15)
    plt.yticks(fontproperties='Times New Roman', size = 15)
    industry_name = CITICIndustryName.query(f"S_INFO_WINDCODE == '{tmp_industry}'").S_INFO_WINDNAME.values[0]
    plt.legend([f"Index Value of {industry_name}"], loc = 'upper left',fontsize = 12.5,shadow = "gray",fancybox = False)
    plt.savefig(f"./Plots/IndCongestionFromStkMean/{tmp_industry[:8]}",bbox_inches = 'tight')
    plt.close()

#### 5.统计胜率

In [22]:
winning_times = pd.DataFrame(index = ["Absolute","Relative"],columns = industry_mahalanobis_distance.columns,data = 0)
signal_times = pd.DataFrame(index = ["Absolute","Relative"],columns = industry_mahalanobis_distance.columns,data = 0)

In [23]:
benchmark = pd.read_excel("./Data/BenchmarkIndex.xlsx",index_col = 0).CLOSE
benchmark.index = pd.to_datetime(benchmark.index)
benchmark = benchmark.loc[start_date:end_date]

In [25]:
for tmp_industry in CITICIndustryIndexClose.columns:
    for date in CITICIndustryIndexClose.index:
        if industry_mahalanobis_distance.at[date,tmp_industry] > 12:
            # trigger times
            signal_times.at["Absolute",tmp_industry] += 1
            signal_times.at["Relative",tmp_industry] += 1
            # absolute accurate
            if CITICIndustryIndexClose.loc[date:,tmp_industry].iloc[:60].iloc[-1]<CITICIndustryIndexClose.loc[date:,tmp_industry].iloc[:60].iloc[0]:
                winning_times.at["Absolute",tmp_industry] += 1
            # relative accurate
            if benchmark.loc[date:].iloc[:60].iloc[-1]/benchmark.loc[date:].iloc[:60].iloc[0]>\
            CITICIndustryIndexClose.loc[date:,tmp_industry].iloc[:60].iloc[-1]/CITICIndustryIndexClose.loc[date:,tmp_industry].iloc[:60].iloc[0]:
                winning_times.at["Relative",tmp_industry] += 1

In [26]:
winning_percentage = winning_times/signal_times

In [27]:
winning_percentage.to_excel("./Output/WinningPctFromStkMean.xlsx")